In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from DataLoader import DataLoader
import datetime
import random
import warnings
from BehaviourFeatureExtractor import compute_speed, compute_head_angle_to_pup, compute_distance_to_pup, extract_base_parameters, plot_mouse_angle_to_pup, convert_seconds_to_frame, extract_trial_from_DLC

### Quality Check analysis (low likelihoods) overview ###

#### Loading and processing all trials from all days #####

In [ ]:
data_dir = "data"
DL = DataLoader(data_dir)

Found items ['.DS_Store', 'MBI5335RR'] at directory data
Found items ['.DS_Store', 'MBI5335RR_240309_d6', 'MBI5335RR_240305_d2', 'MBI5335RR_240304_d1', 'MBI5335RR_240308_d5', 'MBI5335RR_240306_d3', 'MBI5335RR_240307_d4'] at directory data/MBI5335RR
Found items ['MBI5335RR_240309_d6_Avisoft', 'MBI5335RR_240309_d6_Behavior'] at directory data/MBI5335RR/MBI5335RR_240309_d6
Found items ['MBI5335RR_240309_d6_Avisoft_concatenated 2024-04-21  2_08 PM_Stats_SYNCCORRECTED.xlsx'] at directory data/MBI5335RR/MBI5335RR_240309_d6/MBI5335RR_240309_d6_Avisoft
Found items ['MBI5335RR_d6DLC_resnet50_ESM-maternal-learningJun18shuffle1_100000.csv', 'MBI5335RR_240309_d6_BehavSummary.csv'] at directory data/MBI5335RR/MBI5335RR_240309_d6/MBI5335RR_240309_d6_Behavior
Found items ['MBI5335RR_240305_d2_Behavior', 'MBI5335RR_240305_d2_Avisoft'] at directory data/MBI5335RR/MBI5335RR_240305_d2
Found items ['MBI5335RR_240305_d2_BehavSummary.csv', 'MBI5335RR_d2DLC_resnet50_ESM-maternal-learningJun18shuffle1_100000.

In [ ]:
days = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6']
full_data = {}
# get data for d1 experiment
for day in days:
    data = DL.get_data_for_experiment(mouse_id = "MBI5335RR",
                                        day = day)

    full_data[day] = data

#### Plotting low likelihoods, visual assessment ####

In [ ]:
df_DLC, df_summary = full_data['d1']['Behavior']['df_DLC'], full_data['d1']['Behavior']['df_summary']
trial_1_DLC = extract_trial_from_DLC(df_DLC, 1)

# plot mouse and pup positions
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# mask low likelihoods
threshold = 0.3
mask = trial_1_DLC["nose_likelihood"] < threshold


xlim, ylim =  max(df_DLC['msTop_x'].max(), df_DLC['pup_x'].max()), max(df_DLC['msTop_y'].max(), df_DLC['pup_y'].max())

# trial_1_DLC.plot(x='msTop_x', y='msTop_y', style='o', ax=ax, xlim=(0, xlim), ylim=(0, ylim), color = 'r')
trial_1_DLC.plot(x='earRight_x', y='earRight_y', style='o', ax=ax,xlim=(0, xlim), ylim=(0, ylim), color = 'green')
trial_1_DLC.plot(x='earLeft_x', y='earLeft_y', style='o', ax=ax,xlim=(0, xlim), ylim=(0, ylim), color = 'purple')
trial_1_DLC.plot(x='nose_x', y='nose_y', style='o', ax=ax,xlim=(0, xlim), ylim=(0, ylim), color = 'orange')

# trial_1_DLC.plot(x='pup_x', y='pup_y', style='o', ax=ax,xlim=(0, xlim), ylim=(0, ylim), color = 'b')
trial_1_DLC[mask].plot(x='nose_x', y='nose_y', style='o', ax=ax,xlim=(0, xlim), ylim=(0, ylim), color = 'black')